# Тут мы обучаем нашу нейронную сеть что бы потом могли понимать наши жесты

## Библиотеки

In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

## Параметры

In [2]:
# с какой дерриктории будем считывать
DATA_DIR = './data/data.pickle'
URL_MODEL = './model/model.p'

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

## Считываем набор данных

In [3]:
model_dict = pickle.load(open(URL_MODEL, 'rb'))
model = model_dict['model']
cap = cv2.VideoCapture(0)

# Пробуем наш результат

In [4]:
while True:
    data_aux = []
    x_ = []
    y_ = []
    ret, frame = cap.read()
    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks, # координаты ключевых точек руки
                mp_hands.HAND_CONNECTIONS, # для отрисовки связей между точками
                mp_drawing_styles.get_default_hand_landmarks_style(), # возвращает объект, представляющий стиль
                mp_drawing_styles.get_default_hand_connections_style() # для отрисовки связей между ключевыми точками.
            )
        
        # обнаружены ли ключевые точки руки на кадре
        for hand_landmarks in results.multi_hand_landmarks:
            # собираем координаты X и Y всех ключевых точек руки
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                x_.append(x)
                y_.append(y)

            # нормализуем координаты точек относительно минимальных значений
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        # вычисляем координаты ограничивающего прямоугольника баундбокс     
        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        # предсказываем символ по собранным данным
        prediction = model.predict([np.asarray(data_aux)])
        predicted_character = prediction[0]
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        
    key = cv2.waitKey(10)
    if key == ord('q'):
        break

    cv2.putText(frame, 'Press "Q" to close', (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1,
        cv2.LINE_AA)
    cv2.imshow('frame', frame)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()